필요한 클래스, 라이브러리

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import os
import torch
import warnings
from transformers import BertModel, BertTokenizer
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
os.environ['OMP_NUM_THREADS'] = '1'


okt=Okt()
vectorizer=TfidfVectorizer()

웹 크롤링해서 Df 만들기

In [2]:
url="https://www.safetydata.go.kr/disaster-data/disasterNotification?searchStartDttm=&searchEndDttm=&keyword=&orderBy=&currentPage=1&cntPerPage=12000&pageSize=12000"

response=requests.get(url)

if response.status_code == 200:
    file_path = "disaster_message.html"

    with open(file_path, "w", encoding="utf-8") as file:
        file.write(response.text)

    print(f"HTML content has been downloaded and saved to {file_path}.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

filename="disaster_message.html"
html=""
with open (filename, 'r', encoding='UTF-8') as file:
    for line in file:
        html+=line

soup=BeautifulSoup(html,'lxml')

rows=soup.find_all('tr')
data_list=[]

for row in rows:
    cols=row.find_all('td')
    cols=[ele.text.strip() for ele in cols]
    data_list.append(cols)

df=pd.DataFrame(data_list, columns=['number','text','time'])

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(df.head())

HTML content has been downloaded and saved to disaster_message.html.
  number  \
0   None   
1  11993   
2  11992   
3  11991   
4  11990   

                                                                                          text  \
0                                                                                         None   
1                        폭염경보 발효중. 야외 활동을 자제하세요, 충분히 물을 마시고 그늘에서 휴식을 취하는 등 건강관리에 유의하세요. [함평군]N   
2  폭염경보 발효 중입니다. 물을 자주 마시고 개인 건강에 유의하시기 바라며, 야외활동을 자제하고 현장 근로자들은 무더운 낮시간에 휴식하여 주시기 바랍니다.[창원시]N   
3      폭염경보 발효 중. 날이 더운 한낮에는 야외활동(논, 밭, 공사장 등)을 자제하시고 충분한 수분 섭취 및 휴식으로 건강관리에 유의하시기 바랍니다.[해남군]N   
4  보성군 폭염경보 발효 중▲논·밭·건설현장 야외활동 자제▲이웃간 안부확인▲무더위쉼터 이용▲폭염 안전수칙(물, 그늘, 휴식)준수 등 건강관리에 유의바랍니다.[보성군]N   

                  time  
0                 None  
1  2024/07/29 11:45:41  
2  2024/07/29 11:29:36  
3  2024/07/29 11:29:06  
4  2024/07/29 11:19:41  


위치(발송처) 정보 분리

In [ ]:
df=df.drop(0)
df=df.reset_index(drop=True)

df=df.dropna()
df = df.drop_duplicates()
df['number']=pd.to_numeric(df['number'])
df = df.sort_values(by='number', ascending=True)
df = df.set_index('number')

def extract_location(text):
    if text is None:
        return text, 'NaN'
    start=text.rfind('[')
    end=text.rfind(']')
    if start!=-1 and end!=-1:
        location=text[start+1:end]
        text=text[:start]+text[end+1:]
    else:
        location='NaN'
    return text.strip(), location

df[['text', 'location']] = df['text'].apply(lambda x: pd.Series(extract_location(x)))

pd.set_option('display.max_colwidth', None)

print("Df has completed.")